# Учебный проект по А/B-тестированию

***Проведение оценки результатов A/B-теста***

Задача проекта:

- Оценить корректность проведения теста;
- Проанализировать результаты теста.

Исходные данные:

A/B-тест был выполнен на основании технического задания:

- Название теста: recommender_system_test;
- Группы: А (контрольная), B (новая платёжная воронка);
- Дата запуска: 2020-12-07;
- Дата остановки набора новых пользователей: 2020-12-21;
- Дата остановки: 2021-01-04;
- Аудитория: 15% новых пользователей из региона EU;
- Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- Ожидаемое количество участников теста: 6000.
- Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 10%:
- конверсии в просмотр карточек товаров —
    событие product_page
    просмотры корзины — product_cart
    покупки — purchase.
    
    
Целью выполнения данной учебной работы является ***исследование предоставленных данных***  и ***анализ результатов*** теста на основании анализа этих данных.     

Исходные файлы предоставлены Яндекс.Практикум и находятся по адресам:

https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv

https://code.s3.yandex.net/datasets/final_ab_new_users.csv

https://code.s3.yandex.net/datasets/final_ab_events.csv

https://code.s3.yandex.net/datasets/final_ab_participants.csv


## Загрузка библиотек и данных

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import math as mth
import plotly.express as px
from datetime import datetime, timedelta
import datetime
import math
import scipy.stats as stats

In [ ]:
try:
    ab_project_marketing_events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv')
    final_ab_new_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv')
    final_ab_events = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv')
    final_ab_participants = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv')
except:
    ab_project_marketing_events = pd.read_csv('ab_project_marketing_events.csv')
    final_ab_new_users = pd.read_csv('final_ab_new_users.csv')
    final_ab_events = pd.read_csv('final_ab_events.csv')
    final_ab_participants = pd.read_csv('final_ab_participants.csv')

## Подготовка данных

In [ ]:
ab_project_marketing_events.head(100)

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


In [ ]:
final_ab_new_users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


In [ ]:
final_ab_events.head()

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


In [ ]:
final_ab_participants.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


Немного автоматизируем процесс:

In [ ]:
ds_name = ['ab_project_marketing_events', 'final_ab_new_users', 'final_ab_events', 'final_ab_participants']
df_name = [ab_project_marketing_events, final_ab_new_users, final_ab_events, final_ab_participants]
i=0
for t in [ab_project_marketing_events, final_ab_new_users, final_ab_events, final_ab_participants]:
    display()
    display(ds_name[i])
    display()
    t.info()
    display(t.sample(1))
    display('наличие дубликатов - ', t.duplicated().sum())
    display('наличие пропусков - ', t.isnull().sum())
    i=i+1
    display('________________________________')

'ab_project_marketing_events'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


,name,regions,start_dt,finish_dt
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


'наличие дубликатов - '

0

...

'наличие дубликатов - '

0

'наличие пропусков - '

user_id    0
group      0
ab_test    0
dtype: int64

'________________________________'

В датасете `final_ab_events` есть пропуски.

final_ab_events.details - пропущенные значения - 0, то есть отсутсттвие покупок.

In [ ]:
final_ab_events.details = final_ab_events.details.fillna(0)

Требуются преобразование типов в двух колонках c `object` на `datetime64`:

2   `start_dt`   14 non-null     `object`

3   `finish_dt`  14 non-null     `object`

In [ ]:
ab_project_marketing_events['start_dt']= pd.to_datetime(ab_project_marketing_events['start_dt'])
ab_project_marketing_events['finish_dt']= pd.to_datetime(ab_project_marketing_events['finish_dt'])

Требуются преобразование типов в одной колонке c `object` на `datetime64`:

1   `first_date`  61733 non-null  `object`

In [ ]:
final_ab_new_users['first_date']= pd.to_datetime(final_ab_new_users['first_date'])

Требуются преобразование типов в одной колонке c `object` на `datetime64`:

1   `event_dt`    440317 non-null  `object`

In [ ]:
final_ab_events['event_dt']= pd.to_datetime(final_ab_events['event_dt'])

***Проверка после исправления***

In [ ]:
i=0
for t in [ab_project_marketing_events, final_ab_new_users, final_ab_events, final_ab_participants]:
    display()
    display(ds_name[i])
    display()
    display(t.info())
    display(t.sample(1))
    display('наличие дубликатов - ', t.duplicated().sum())
    display('наличие пропусков - ', t.isnull().sum())
    i=i+1
    display('________________________________')

'ab_project_marketing_events'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


None

,name,regions,start_dt,finish_dt
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03


'наличие дубликатов - '

...

'наличие дубликатов - '

0

'наличие пропусков - '

user_id    0
group      0
ab_test    0
dtype: int64

'________________________________'

### Промежуточные выводы

Исходные данные приведены в соответствие правилам Python, типы соответствуют названиям.

### Расшифровка данных в датасетах

Посмотрим на качество данных в каждом датасете:

In [ ]:
ab_project_marketing_events

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


Это был -***Календарь маркетинговых событий на 2020 год***

Структура файла:

- name — название маркетингового события;
- regions — регионы, в которых будет проводиться рекламная кампания;
- start_dt — дата начала кампании;
- finish_dt — дата завершения кампании.


In [ ]:
final_ab_new_users

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
61728,1DB53B933257165D,2020-12-20,EU,Android
61729,538643EB4527ED03,2020-12-20,EU,Mac
61730,7ADEE837D5D8CBBD,2020-12-20,EU,PC
61731,1C7D23927835213F,2020-12-20,EU,iPhone


Это были - ***Все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года***

Структура файла:

- user_id — идентификатор пользователя;
- first_date — дата регистрации;
- region — регион пользователя;
- device — устройство, с которого происходила регистрация.

фактически сложенные в один файл профили участников теста

In [ ]:
final_ab_events

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
...,...,...,...,...
440312,245E85F65C358E08,2020-12-30 19:35:55,login,0.00
440313,9385A108F5A0A7A7,2020-12-30 10:54:15,login,0.00
440314,DB650B7559AC6EAC,2020-12-30 10:59:09,login,0.00
440315,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,0.00


Это были - ***Все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года***

Структура файла:

- user_id — идентификатор пользователя;
- event_dt — дата и время события;
- event_name — тип события;
- details — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.


In [ ]:
final_ab_participants

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test
...,...,...,...
18263,1D302F8688B91781,B,interface_eu_test
18264,3DE51B726983B657,A,interface_eu_test
18265,F501F79D332BE86C,A,interface_eu_test
18266,63FBE257B05F2245,A,interface_eu_test


Это была - ***Tаблица участников тестов***

Структура файла:

- user_id — идентификатор пользователя;
- b_test — название теста;
- group — группа пользователя.


## Соответствие данных требованиям технического задания.

***Проверяем***

- Название теста: recommender_system_test;
- Группы: А (контрольная), B (новая платёжная воронка);
- Дата запуска: 2020-12-07;
- Дата остановки набора новых пользователей: 2020-12-21;
- Дата остановки: 2021-01-04;
- Аудитория: 15% новых пользователей из региона EU;
- Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- Ожидаемое количество участников теста: 6000.
- Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 10%:
- конверсии в просмотр карточек товаров — событие product_page
- просмотры корзины — product_cart
- покупки — purc

Проверяем корректность его проведения и проанализируем полученные результаты


### Проверка Участников теста `recommender_system_test`.

Список участников всех тестов находятся в переменной `final_ab_participants` - Таблица участников тестов

Выделим в отдельный датасет `recommender_system_test` участников эксперимента

In [ ]:
recommender_system_test = final_ab_participants[final_ab_participants['ab_test'] == 'recommender_system_test']
recommender_system_test = recommender_system_test.drop('ab_test', axis=1)
recommender_system_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6701 entries, 0 to 6700
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  6701 non-null   object
 1   group    6701 non-null   object
dtypes: object(2)
memory usage: 157.1+ KB


In [ ]:
# Все ли user_id уникальны?
recommender_system_test.user_id.nunique()

6701

In [ ]:
# Какие группы есть?
recommender_system_test.group.unique()

array(['A', 'B'], dtype=object)

In [ ]:
# Сколько участников в каждой группе?
t = recommender_system_test.groupby(by='group').agg('count')
t['ratio'] = t['user_id'] / t['user_id'].sum()
t

,user_id,ratio
group,,
A,3824,0.570661
B,2877,0.429339


Группы различаются более, чем на 10%. Новая платежная воронка меньше на 25% контрольной группы.

в переменной `test = recommender_system_test.user_id` сохраняю временный список `user_id` участвующих в тесте `recommender_system_test`.

In [ ]:
test = recommender_system_test.user_id
test

0       D1ABA3E2887B6A73
1       A7A3664BD6242119
2       DABC14FDDFADD29E
3       04988C5DF189632E
4       482F14783456D21B
              ...       
6696    053FB26D6D49EDDC
6697    9D263B8EF15CF188
6698    F2FBBA33F37DEC46
6699    29C92313A98B1176
6700    6715343AFBA285AE
Name: user_id, Length: 6701, dtype: object

### Проверяем дату запуска теста

Были ли зарегистрированы пользователи, участвующие в тесте в период проведения теста?
Список новых пользователей лежит в переменной `final_ab_new_users`

In [ ]:
# Сколько было зарегистрировано в интернет-магазине в период с 7 по 21 декабря 2020 года пользователей из списка теста?
new_users = final_ab_new_users[final_ab_new_users.user_id.isin(test)]
final_ab_new_users.user_id.isin(test).sum()

6701

In [ ]:
new_users

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
13,E6DE857AFBDC6102,2020-12-07,EU,PC
20,DD4352CDCF8C3D57,2020-12-07,EU,Android
23,831887FE7F2D6CBA,2020-12-07,EU,Android
39,4CB179C7F847320B,2020-12-07,EU,iPhone
...,...,...,...,...
61685,A09DFE27D4CFF8E8,2020-12-20,EU,Android
61693,0F7D49FC184EDCDE,2020-12-20,EU,PC
61709,574ACBC674BC385D,2020-12-20,EU,Mac
61710,0416B34D35C8C8B8,2020-12-20,EU,Android


In [ ]:
# Как эти участники теста распределились по датам?

In [ ]:
min_date_reg = new_users.first_date.min()
display('Минимальная дата регистрации - ', min_date_reg)
max_date_reg= new_users.first_date.max()
display('Максимальная дата регистрации - ', max_date_reg)
delta_day = (max_date_reg - min_date_reg)# .dt.days
display(delta_day)

'Минимальная дата регистрации - '

Timestamp('2020-12-07 00:00:00')

'Максимальная дата регистрации - '

Timestamp('2020-12-21 00:00:00')

Timedelta('14 days 00:00:00')

Начало и конец регистрации участников эксперимента соответствует техническому заданию.

Посмотрим распределение времени регистрации по датам:

In [ ]:
new_users.sample(1)

,user_id,first_date,region,device
3625,0E1FEA52E890B36A,2020-12-07,EU,Android


In [ ]:
t = new_users.groupby(by = 'first_date', as_index=False).agg({'user_id':"count"})


In [ ]:
# функция для вывода гистограммы, чтобы однообразно выводить графики.
def my_bar(df, x, y, title, y_title):
    fig = px.bar(df, x=x, y=y)
    fig.update_xaxes(tickangle=-90, tickmode = 'linear')

    fig.update_layout(title=title,
                      yaxis_title=y_title,
                      margin=dict(l=0, r=0))
    fig.show()

In [ ]:
my_bar(t, t['first_date'], t['user_id'],
      'Распределение количества событий при регистрации', 'Количество событий')

Участники регистрировались неравномерно, особенно в последний день регистрации.

### Проверка даты остановки эксперимента (наблюдения) `recommender_system_test`.

In [ ]:
#Проверим дату остановки эксперимента (наблюдения).

#Список всех событий у нас лежит в переменной `final_ab_events`.

In [ ]:
events = final_ab_events[final_ab_events.user_id.isin(test)]
final_ab_events.user_id.isin(test).sum()

24698

In [ ]:
# переменная хранящая события участников эксперимента
events

,user_id,event_dt,event_name,details
5,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99
17,3C5DD0288AC4FE23,2020-12-07 19:42:40,purchase,4.99
58,49EA242586C87836,2020-12-07 06:31:24,purchase,99.99
71,2B06EB547B7AAD08,2020-12-07 21:36:38,purchase,4.99
74,A640F31CAC7823A6,2020-12-07 18:48:26,purchase,4.99
...,...,...,...,...
440239,D21F0D4FDCD82DB2,2020-12-29 02:17:00,login,0.00
440258,96BDD55846D1F7F6,2020-12-29 16:53:42,login,0.00
440267,553BAE96C6EB6240,2020-12-29 14:09:14,login,0.00
440281,DD4352CDCF8C3D57,2020-12-30 12:42:56,login,0.00


In [ ]:
t = events.copy()

In [ ]:
t['event_dt_day'] = t.event_dt.dt.date
t

,user_id,event_dt,event_name,details,event_dt_day
5,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99,2020-12-07
17,3C5DD0288AC4FE23,2020-12-07 19:42:40,purchase,4.99,2020-12-07
58,49EA242586C87836,2020-12-07 06:31:24,purchase,99.99,2020-12-07
71,2B06EB547B7AAD08,2020-12-07 21:36:38,purchase,4.99,2020-12-07
74,A640F31CAC7823A6,2020-12-07 18:48:26,purchase,4.99,2020-12-07
...,...,...,...,...,...
440239,D21F0D4FDCD82DB2,2020-12-29 02:17:00,login,0.00,2020-12-29
440258,96BDD55846D1F7F6,2020-12-29 16:53:42,login,0.00,2020-12-29
440267,553BAE96C6EB6240,2020-12-29 14:09:14,login,0.00,2020-12-29
440281,DD4352CDCF8C3D57,2020-12-30 12:42:56,login,0.00,2020-12-30


In [ ]:
t = t.groupby(by = 'event_dt_day', as_index=False).agg({'user_id':"count"})
t

,event_dt_day,user_id
0,2020-12-07,709
1,2020-12-08,593
2,2020-12-09,746
3,2020-12-10,613
4,2020-12-11,542
5,2020-12-12,558
6,2020-12-13,504
7,2020-12-14,1355
8,2020-12-15,1316
9,2020-12-16,1454


In [ ]:
## Распределение по датам
min_date = events.event_dt.min()
display('Минимальная дата события - ', min_date)
max_date = events.event_dt.max()
display('Максимальная дата события - ', max_date)
delta_day = (max_date - min_date)# .dt.days
display(delta_day)
# Гистограмма
my_bar(t, t['event_dt_day'], t['user_id'],
      'Распределение количества событий эксперимента', 'Количество событий')

'Минимальная дата события - '

Timestamp('2020-12-07 00:05:57')

'Максимальная дата события - '

Timestamp('2020-12-30 12:42:57')

Timedelta('23 days 12:37:00')

Последним днем наблюдений явилось 2020-12-30. Хотя по техническому заданию должны были наблюдать до - 2021-01-04.

В последний день очень мало событий. После завершения проверки соответствия техническго задания исключим их.

In [ ]:
f = events.event_dt.dt.date

In [ ]:
f.unique()

array([datetime.date(2020, 12, 7), datetime.date(2020, 12, 8),
       datetime.date(2020, 12, 9), datetime.date(2020, 12, 10),
       datetime.date(2020, 12, 11), datetime.date(2020, 12, 12),
       datetime.date(2020, 12, 13), datetime.date(2020, 12, 14),
       datetime.date(2020, 12, 15), datetime.date(2020, 12, 16),
       datetime.date(2020, 12, 17), datetime.date(2020, 12, 18),
       datetime.date(2020, 12, 19), datetime.date(2020, 12, 20),
       datetime.date(2020, 12, 21), datetime.date(2020, 12, 22),
       datetime.date(2020, 12, 23), datetime.date(2020, 12, 24),
       datetime.date(2020, 12, 25), datetime.date(2020, 12, 26),
       datetime.date(2020, 12, 27), datetime.date(2020, 12, 28),
       datetime.date(2020, 12, 29), datetime.date(2020, 12, 30)],
      dtype=object)

### Распределение по видам событий

In [ ]:
t = events.groupby(by='event_name')['user_id'].agg('unique')#.sort_values(ascending=False)
tt = events.groupby(by='event_name', as_index=False)['user_id'].agg('nunique').sort_values(by='user_id', ascending=False)
tt

,event_name,user_id
0,login,3675
2,product_page,2303
3,purchase,1128
1,product_cart,1079


In [ ]:
#t.hist(bins = 4, figsize=[18, 8]);

In [ ]:
my_bar(tt, tt['event_name'], tt['user_id'],
      'Распределение количества событий эксперимента между событиями', 'Количество событий')

### Проверка пользователей на регион

In [ ]:
#Сколько всего пользователей в регионе EU?
final_ab_new_users[final_ab_new_users['region'] == 'EU'].shape[0]

46270

In [ ]:
#Все ли пользователи из региона EU?
t = final_ab_new_users.query('region == "EU"')
final_ab_new_users.query('region == "EU"').shape[0]

46270

In [ ]:
#Сколько пользователей в регионе EU было в период проведения теста (регистрации)?
t = t[t['first_date'] >= min_date_reg]
t = t[t['first_date'] <= max_date_reg]
t.shape[0]

42340

In [ ]:
final_ab_new_users

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
61728,1DB53B933257165D,2020-12-20,EU,Android
61729,538643EB4527ED03,2020-12-20,EU,Mac
61730,7ADEE837D5D8CBBD,2020-12-20,EU,PC
61731,1C7D23927835213F,2020-12-20,EU,iPhone


In [ ]:
i0 = final_ab_new_users.user_id.isin(test).sum() # количество участвующих в тесте
i0

6701

In [ ]:
#количество участвующих в тесте из региона EU
i1 = final_ab_new_users[final_ab_new_users.user_id.isin(test)].query('region == "EU"').shape[0] #количество участвующих в тесте из региона EU
i1

6351

In [ ]:
print('В тест `recommender_system_test` попало ', i0-i1, 'пользователей из других регионов')

В тест `recommender_system_test` попало  350 пользователей из других регионов


После завершения проверки соответствия технического задания исключим их.

In [ ]:
# Cколько процентов новых пользователей вошло в тест
#print ('В тест `recommender_system_test` было отобрано только ', round (i0/t.shape[0]*100,2), '% пользователей из региона EU вместо 15%')

In [ ]:
# Cколько процентов новых пользователей вошло в тест
print ('В тест `recommender_system_test` было отобрано только ', round (i1/t.shape[0]*100,2), '% пользователей из региона EU вместо 15%')

В тест `recommender_system_test` было отобрано только  15.0 % пользователей из региона EU вместо 15%


### Пригодность метрик

По техническому заданию тест должен подтвердить или опровергнуть улучшение метрик не менее чем на 10 %:

- конверсии в просмотр карточек товаров — событие product_page
- просмотры корзины — product_cart
- покупки — purchase

***Обсужнение метрик***

**конверсии в просмотр карточек товаров** — событие product_page годная метрика - легко контролируется при изменении количества участников в контрольной и экспериментальной группе.

**просмотры корзины** — product_cart  Негодная метрика -  метрика в такой формулировке зависит от количества посетителей в группе. При изменении количества посетителей в группе будет меняться. Рекомендация контролировать удельные просмотры, скажем от числа посетителей в группе.

**покупки** — purchase тоже малопригодная метрика плохо сформулированная. При изменении количества посетителей в группе будет меняться. Рекомендация контролировать удельные просмотры, скажем от числа посетителей в группе.

#### Выводы

Данные подготовленные для теста не соответствуют техническому заданию для A/B тестирования. Выявлены следующие замечания и наблюдения:

- В тесте рассматривались 6701 новый пользователь;
- Группы А и В ренавномерны. Группы различаются более, чем на 10%. Новая платежная воронка меньше на 25% контрольной группы;
- Начало и конец регистрации участников эксперимента соответствует техническому заданию;
- Участники регистрировались неравномерно, особенно в последний день регистрации;
- Последним днем наблюдений явилось 2020-12-30. Хотя по техническому заданию должны были наблюдать до - 2021-01-04;
- В последний день очень мало событий;
- В тест `recommender_system_test` попало  350 пользователей из других регионов;
- В тест `recommender_system_test` было отобрано только  15 % пользователей из региона EU, как и положено по ТЗ - 15%;
- Две из трех метрик, контролируемых в `recommender_system_test` непригодны для использования и требуют переформулировки.


## Оценка корректности проведения теста

### Время проведения теста

Обращает на себя внимание, несколько заметных притоков пользователей на графиках (гистограммах) распределения пользователей по датам. Ответ можно найти в датасете /datasets/ab_project_marketing_events.csv — календарь маркетинговых событий на 2020 год.

В период 2020-12-25 по 2021-01-03 проводиться акция Christmas&New Year Promo в регионах	EU, N.America. Сама по себе рекламная (маркетинговая) акция на проведение А/В теста не влияет, как как маркетинговые мероприятия действуют и на экспериментальную и на контрольную группу. Но акция может поменять абсолютные значения метрик (конверсий), что впоследствии может сказаться на принятии решений.

### Аудиторию теста

Проверим  нет ли пересечения аудитории теста `recommender_system_test` с другими тестами

Очистим аудиторию теста `recommender_system_test` от излишек замеченных в прошлом разделе:



In [ ]:
final_ab_new_users
final_ab_new_users.region.unique()

array(['EU', 'N.America', 'APAC', 'CIS'], dtype=object)

In [ ]:
test = final_ab_new_users[final_ab_new_users.user_id.isin(test)].query('region == "EU"')['user_id']
test # Обновленный список user_id, подходящий для A|B теста `recommender_system_test

0        D72A72121175D8BE
13       E6DE857AFBDC6102
20       DD4352CDCF8C3D57
23       831887FE7F2D6CBA
39       4CB179C7F847320B
               ...       
61685    A09DFE27D4CFF8E8
61693    0F7D49FC184EDCDE
61709    574ACBC674BC385D
61710    0416B34D35C8C8B8
61715    89CB0BFBC3F35126
Name: user_id, Length: 6351, dtype: object

### Проверка участников теста на участие в других тестах

Необходимо проверить нет ли среди участников теста `recommender_system_test` пользователей, принявших участие в других тестах. Это важно потому, что нам не известно, что им в тех тестах показывали, и возможно их поведение может быть изменено под воздействием тех показов. При наличии их нужно исключить.

In [ ]:
final_ab_participants

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test
...,...,...,...
18263,1D302F8688B91781,B,interface_eu_test
18264,3DE51B726983B657,A,interface_eu_test
18265,F501F79D332BE86C,A,interface_eu_test
18266,63FBE257B05F2245,A,interface_eu_test


In [ ]:
t = final_ab_participants[final_ab_participants['user_id'].isin(test)]
t # временный список user_id участников теста
t = t[t['ab_test'] !='recommender_system_test']
t # список участников теста, участвующих в других тестах

,user_id,group,ab_test
6702,773ECB64E45DEBAB,A,interface_eu_test
6704,AABA4219186465C9,A,interface_eu_test
6705,2BA8FA8754D1FE50,B,interface_eu_test
6706,FB6F6BC119E1DBD5,B,interface_eu_test
6715,0ABA6D32AE7AFB9B,A,interface_eu_test
...,...,...,...
18227,EF2E4FAF26951358,A,interface_eu_test
18245,2B0CD24EE4291CA0,B,interface_eu_test
18252,CA6F4DAED160E5B1,A,interface_eu_test
18257,C70DF942E542DA69,A,interface_eu_test


In [ ]:
test

0        D72A72121175D8BE
13       E6DE857AFBDC6102
20       DD4352CDCF8C3D57
23       831887FE7F2D6CBA
39       4CB179C7F847320B
               ...       
61685    A09DFE27D4CFF8E8
61693    0F7D49FC184EDCDE
61709    574ACBC674BC385D
61710    0416B34D35C8C8B8
61715    89CB0BFBC3F35126
Name: user_id, Length: 6351, dtype: object

In [ ]:
# убрать лишних
test =test[~test.isin(t['user_id'])]
test

0        D72A72121175D8BE
23       831887FE7F2D6CBA
39       4CB179C7F847320B
65       16C69ED14DA9F4A8
69       3C5DD0288AC4FE23
               ...       
61681    3CA972F86411CF13
61693    0F7D49FC184EDCDE
61709    574ACBC674BC385D
61710    0416B34D35C8C8B8
61715    89CB0BFBC3F35126
Name: user_id, Length: 4749, dtype: object

### Выводы и наблюдения

Продолжаем улучшать данные  А/В теста `final_ab_participants`:
- приняли решение, что наличие маркетингового мероприятия во время сбора информации для А/В тесте не повлияет на результаты, так как эти мероприятия действуют и на экспериментальную группу и на контрольную
- исключили участников теста(пользователей) не из региона проведения теста:
- убрали из списка участников теста, участвующих в других тестах.

Проверим также на пересечение пользователей внутри теста. Я это сделал в конце пункта работы 5.2, когда разделил датасеты.



### Проверка ТЗ в части исключения пользователя из наблюдения после 14го дня после регистрации

Техническое задание предусматривает - Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 10%

Следовательно наблюдения и замер метрик необходимо производить 14 дней, а затем участника эксперимента исключать из группы.

Первые посещения приложения участниками у нас в датасете final_ab_new_users

In [ ]:
final_ab_new_users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


In [ ]:
final_ab_new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     61733 non-null  object        
 1   first_date  61733 non-null  datetime64[ns]
 2   region      61733 non-null  object        
 3   device      61733 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ MB


In [ ]:
# формируем временный датасет
t = final_ab_new_users.copy()
# добавляем столбец последнего дня наблюдений
t['last_date'] = t['first_date'] + datetime.timedelta(days=14)
# убираем лишние колонки
t = t.drop(columns=['first_date', 'region', 'device'])
# добавляем последний день наблюдений в датасет событий теста
events = events.merge(t, left_on='user_id', right_on='user_id')
# фильтруем события
events = events[events['event_dt'] <= events['last_date']]
# считаем оставшихся пользователей
events['user_id'].nunique()
# корректируем список участников эксперимента - теста
test = events['user_id'].unique()
test

array(['831887FE7F2D6CBA', '3C5DD0288AC4FE23', '49EA242586C87836', ...,
       'BEF16764A13AEC34', '1E6B9298415AA97A', '23DDD27AC3FEFA63'],
      dtype=object)

In [ ]:
len (test)

3675

### Дополнительный вывод

Исключив события пользователей после 14 дня после регистрации, потеряли еще некоторое количество уникальных участников эксперимента. Остаток 3675 пользователей.

## Исследовательский анализ данных

Мы окончательно определились с количеством и списком участников в обновленном А/В тесте, поэтому можно объеденить датасеты для удобства предварительного анализа и дальнейшего проведения Z-теста.

итак:
- список пользователей - test
- Даты регистрации новых участников - с min_date_reg = 2020-12-07 по max_date_reg = 2020-12-21. всего 14 дней.
- даты наблюдения - min_date =2 020-12-07 по max_date=2020-12-30. всего 22 дня.
- Аудитория: 15% новых пользователей из региона EU;
- Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- Ожидаемое количество участников теста: 3600 (сейчас - 3675).
- Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 10%:


        1. конверсии в просмотр карточек товаров — событие product_page
        2. конверсии в просмотры корзины — событие product_cart
        3. Конверсии в покупки — событие purchase



In [ ]:
events = final_ab_events[final_ab_events.user_id.isin(test)]
print (events)
final_ab_events.user_id.isin(test).sum()

                 user_id            event_dt event_name  details
5       831887FE7F2D6CBA 2020-12-07 06:50:29   purchase     4.99
17      3C5DD0288AC4FE23 2020-12-07 19:42:40   purchase     4.99
58      49EA242586C87836 2020-12-07 06:31:24   purchase    99.99
71      2B06EB547B7AAD08 2020-12-07 21:36:38   purchase     4.99
74      A640F31CAC7823A6 2020-12-07 18:48:26   purchase     4.99
...                  ...                 ...        ...      ...
440239  D21F0D4FDCD82DB2 2020-12-29 02:17:00      login     0.00
440258  96BDD55846D1F7F6 2020-12-29 16:53:42      login     0.00
440267  553BAE96C6EB6240 2020-12-29 14:09:14      login     0.00
440281  DD4352CDCF8C3D57 2020-12-30 12:42:56      login     0.00
440289  75845C83258FBF73 2020-12-30 06:42:52      login     0.00

[24698 rows x 4 columns]


24698

Годных к исследованию осталось совсем мало от того количества, которое было в начале!

### Подготовка данных

In [ ]:
events

,user_id,event_dt,event_name,details
5,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99
17,3C5DD0288AC4FE23,2020-12-07 19:42:40,purchase,4.99
58,49EA242586C87836,2020-12-07 06:31:24,purchase,99.99
71,2B06EB547B7AAD08,2020-12-07 21:36:38,purchase,4.99
74,A640F31CAC7823A6,2020-12-07 18:48:26,purchase,4.99
...,...,...,...,...
440239,D21F0D4FDCD82DB2,2020-12-29 02:17:00,login,0.00
440258,96BDD55846D1F7F6,2020-12-29 16:53:42,login,0.00
440267,553BAE96C6EB6240,2020-12-29 14:09:14,login,0.00
440281,DD4352CDCF8C3D57,2020-12-30 12:42:56,login,0.00


По рекомендации не замещаю пропуски нулями. Эта колонка в расчетах не участвует. Сохраним ее в первозданном виде.

In [ ]:
t = final_ab_participants[final_ab_participants['user_id'].isin(test)]
t # временный список user_id участников теста
# для удобства работы объеденим события и группы
events = events.merge(t, left_on='user_id', right_on='user_id')

И разделим на два датасета, лишние столбцы уберем

In [ ]:
events_a = events[events['group'] == 'A'].drop(columns=['ab_test', 'group', 'details'])
events_a

,user_id,event_dt,event_name
0,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase
1,831887FE7F2D6CBA,2020-12-09 02:19:17,purchase
2,831887FE7F2D6CBA,2020-12-07 06:50:30,product_cart
3,831887FE7F2D6CBA,2020-12-08 10:52:27,product_cart
4,831887FE7F2D6CBA,2020-12-09 02:19:17,product_cart
...,...,...,...
30579,1484BBF124DB1B18,2020-12-27 10:53:34,login
30581,BEF16764A13AEC34,2020-12-21 03:49:49,login
30583,BEF16764A13AEC34,2020-12-22 18:52:25,login
30585,BEF16764A13AEC34,2020-12-24 22:11:00,login


In [ ]:
events_b = events[events['group'] == 'B'].drop(columns=['ab_test', 'group', 'details'])
events_b

,user_id,event_dt,event_name
11,49EA242586C87836,2020-12-07 06:31:24,purchase
12,49EA242586C87836,2020-12-09 15:39:47,purchase
13,49EA242586C87836,2020-12-11 10:06:37,purchase
14,49EA242586C87836,2020-12-12 04:30:41,purchase
15,49EA242586C87836,2020-12-16 09:28:37,purchase
...,...,...,...
30586,1E6B9298415AA97A,2020-12-21 18:20:12,login
30587,1E6B9298415AA97A,2020-12-22 03:22:34,login
30588,1E6B9298415AA97A,2020-12-23 11:34:53,login
30589,1E6B9298415AA97A,2020-12-24 18:45:58,login


Сравним пользователей из 2х датасетов на предмет пересечения

In [ ]:
set(events_b['user_id']).isdisjoint(set(events_a['user_id']))

False

К сожалению, пересечение есть, хоть один пользователь да зашел не на свою картинку приложения.

Cоставим список пересечения двух списков

In [ ]:
result=list(set(events_b['user_id'].unique()) & set(events_a['user_id'].unique()))
len (result)

441

In [ ]:
# Исключаем "пересекантов" из одного списка
events_a = events_a[~events_a.user_id.isin(result)]

In [ ]:
# И из другого
events_b = events_b[~events_b.user_id.isin(result)]

In [ ]:
events_a.user_id.nunique()

2422

In [ ]:
events_b.user_id.nunique()

812

Еще раз скорректировали количество пользователей участвующих в эксперименте. Осталось - сколько осталось!

###  Распределение в выборках количества событий

Рассотрим распределение событий в каждой группе, чтобы понимать, насколько события равноменно распределились по группам.

Между группами даже по количеству события распределены неравномерно. Предполагаю, что это связано с ограничением - "15% из региона EU".

Посмотрим на графиках:

In [ ]:
events_b

,user_id,event_dt,event_name
11,49EA242586C87836,2020-12-07 06:31:24,purchase
12,49EA242586C87836,2020-12-09 15:39:47,purchase
13,49EA242586C87836,2020-12-11 10:06:37,purchase
14,49EA242586C87836,2020-12-12 04:30:41,purchase
15,49EA242586C87836,2020-12-16 09:28:37,purchase
...,...,...,...
30586,1E6B9298415AA97A,2020-12-21 18:20:12,login
30587,1E6B9298415AA97A,2020-12-22 03:22:34,login
30588,1E6B9298415AA97A,2020-12-23 11:34:53,login
30589,1E6B9298415AA97A,2020-12-24 18:45:58,login


In [ ]:
# создадим временную таблицу
t= events_b.copy()
# доп. столбец для дальнейшей сорировки
t['event_dt_day'] = t.event_dt.dt.date
t

,user_id,event_dt,event_name,event_dt_day
11,49EA242586C87836,2020-12-07 06:31:24,purchase,2020-12-07
12,49EA242586C87836,2020-12-09 15:39:47,purchase,2020-12-09
13,49EA242586C87836,2020-12-11 10:06:37,purchase,2020-12-11
14,49EA242586C87836,2020-12-12 04:30:41,purchase,2020-12-12
15,49EA242586C87836,2020-12-16 09:28:37,purchase,2020-12-16
...,...,...,...,...
30586,1E6B9298415AA97A,2020-12-21 18:20:12,login,2020-12-21
30587,1E6B9298415AA97A,2020-12-22 03:22:34,login,2020-12-22
30588,1E6B9298415AA97A,2020-12-23 11:34:53,login,2020-12-23
30589,1E6B9298415AA97A,2020-12-24 18:45:58,login,2020-12-24


In [ ]:
# Группируем, сортируем
t = t.groupby(by='event_dt_day', as_index=False)['user_id'].agg('nunique') #.sort_values(by='user_id', ascending=False)
t

,event_dt_day,user_id
0,2020-12-07,150
1,2020-12-08,103
2,2020-12-09,151
3,2020-12-10,111
4,2020-12-11,76
5,2020-12-12,94
6,2020-12-13,77
7,2020-12-14,115
8,2020-12-15,105
9,2020-12-16,150


In [ ]:
# Строим красивый график
my_bar(t, t['event_dt_day'], t['user_id'],
      'Распределение количества событий эксперимента в группе В', 'Количество событий')

In [ ]:
# тоже для другого набора данных
t= events_a.copy()
t['event_dt_day'] = t.event_dt.dt.date
t
t = t.groupby(by='event_dt_day', as_index=False)['user_id'].agg('nunique') #.sort_values(by='user_id', ascending=False)
t
my_bar(t, t['event_dt_day'], t['user_id'],
      'Распределение количества событий эксперимента в группе А', 'Количество событий')

В группе А скачек активности с 14-12-2021. Этот скачек пока непонятен. Надо посмотреть динамику набора пользователей.

In [ ]:
# events_b.user_id.unique() # - список уникальных пользователей группы В экперимента
events_a.user_id.unique() # - список уникальных пользователей группы А экперимента

array(['831887FE7F2D6CBA', '3C5DD0288AC4FE23', '2B06EB547B7AAD08', ...,
       '64597B30278462F5', '1484BBF124DB1B18', '23DDD27AC3FEFA63'],
      dtype=object)

In [ ]:
t = final_ab_new_users[final_ab_new_users['user_id'].isin(events_a.user_id.unique())]
t

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
23,831887FE7F2D6CBA,2020-12-07,EU,Android
69,3C5DD0288AC4FE23,2020-12-07,EU,PC
105,B27F5380AF0936D3,2020-12-07,EU,Android
130,ABABE1BAB6C77540,2020-12-07,EU,iPhone
...,...,...,...,...
61637,C27F677720E68EA4,2020-12-20,EU,Android
61681,3CA972F86411CF13,2020-12-20,EU,PC
61693,0F7D49FC184EDCDE,2020-12-20,EU,PC
61709,574ACBC674BC385D,2020-12-20,EU,Mac


In [ ]:
my_bar(t, t['first_date'], t['user_id'],
      'Распределение количества регистраций в группе А', 'Количество событий')

Регистрации в группе А резко рванули вверх. А в группе Б?

In [ ]:
t = final_ab_new_users[final_ab_new_users['user_id'].isin(events_b.user_id.unique())]
t
my_bar(t, t['first_date'], t['user_id'],
      'Распределение количества регистраций в группе В', 'Количество событий')

В группе В резкого скачка не наблюдается - регистрация идет очень неравноменрно. Хотя, если приглядеться видна тенденция к росту.

Число событий в выборках распределено по дням в группах неравномерно. В группе В в начале наблюдения излишняя активность пользователей.

В группе А скачек активности с 14-12-2020, который можно связать со стабильным ростом регистрации начиная с 14 декабря. Явного такого же эффекта в группе В не наблюдаем.

### Сравнение активности пользователей разных групп

Проверим среднее количество событий на 1го пользователя в разных группах

In [ ]:
n_a = events_a.shape[0] / events_a.user_id.nunique()
n_a

8.076383154417837

In [ ]:
n_b = events_b.shape[0] / events_b.user_id.nunique()
n_b

6.735221674876847

Экспериментальная группа менее активна

### Сравнение конверсии разных групп

Сначала - общая конверсия всей выборки, чтобы понимать, в каком мире живем.

In [ ]:
conv = events.groupby(by='event_name', as_index=False)['user_id'].nunique()
conv
conv = conv.sort_values(by = 'user_id', ascending=False)
conv['ratio'] = conv['user_id'] / events.user_id.nunique()
conv['rang'] = [1, 2, 4, 3]
conv = conv.sort_values(by = 'rang', ascending=True)

In [ ]:
fig = px.funnel(conv, x='user_id', y='event_name', title ='Воронка событий для всех участников эксперимента')
fig.show()

Воронка событий имеет второй раструб между финальным (целевым) событием - Оплатой и, явно не финальным - просмотром корзины. Это противоречие можно объяснить догадкой, что не все покупатели перед оплатой заходят в корзину посмотреть, а сразу переходят к оплате.

Рассмотрим воронки отдельно по участникам контрольной и экспериментальной групп.

In [ ]:
conv_a = events_a.groupby(by='event_name', as_index=False)['user_id'].nunique()
conv_a
conv_a = conv_a.sort_values(by = 'user_id', ascending=False)
conv_a['ratio'] = conv_a['user_id'] / events_a.user_id.nunique()
conv_a['rang'] = [1, 2, 4, 3]
conv_a = conv_a.sort_values(by = 'rang', ascending=True)
conv_a

,event_name,user_id,ratio,rang
0,login,2422,1.000000,1
2,product_page,1571,0.648637,2
1,product_cart,728,0.300578,3
3,purchase,773,0.319158,4


In [ ]:
fig = px.funnel(conv_a, x='user_id', y='event_name', title ='Воронка событий для группы А участников эксперимента')
fig.show()

In [ ]:
conv_b = events_b.groupby(by='event_name', as_index=False)['user_id'].nunique()
conv_b
conv_b = conv_b.sort_values(by = 'user_id', ascending=False)
conv_b['ratio'] = conv_b['user_id'] / events_b.user_id.nunique()
conv_b['rang'] = [1, 2, 4, 3]
conv_b = conv_b.sort_values(by = 'rang', ascending=True)
conv_b

,event_name,user_id,ratio,rang
0,login,812,1.000000,1
2,product_page,461,0.567734,2
1,product_cart,225,0.277094,3
3,purchase,228,0.280788,4


In [ ]:
fig = px.funnel(conv_b, x='user_id', y='event_name', title ='Воронка событий для группы В участников эксперимента')
fig.show()

Конверсии на всех этапах воронки событий в группе В экспериментальной, ниже конверсий группы А аналогичных этапов.

А/В тест final_ab_participants, результаты которого были представлены для изучения был сделан недостаточно хорошо. Помимо недостатков отмеченных в предыдушем разделе при выполнении исследовательского анализа были выявлены следующие недостатки:
### Выводы и наблюдения

- часть участников эксперимента final_ab_participants была задействована в других экспериментах, что недозволительно, так как на поведение одного участника может оказывать влияние другого эксперимента.
- группы А и В частично пересекались;
- группы А и В при правильной фильтрации получаются очень разными, разница еще больше возрастает до 70 %.
- распределение в группах по дням неравномерное. Возможно на это влияют маркетинговые мероприятия, проводимые в период сбора информации для теста.


## Проведение статистической разницы долей z-критерием

### Подготовка данных и функций

Подготовлено два файла с конверсиями по различным событиям для контрольной и экспериментальной группе. Проверим нулевую теорию, что пропорции одинаковых событий в разных группах одинаковы:

***Н0***: Конверсии событий в группе А ***Равны*** конверсиям соббытий в группе В;

***Н1***: Конверсии событий в группе А ***Отличаются*** от конверсии соббытий в группе В.

Проведем исследование функцией разработанной ранее в рамках изучения курса Аналитик данных. Функция требует небольшой доработки.

In [ ]:
# функция для проверки гипотезы равности долей при переходе от события к событию

def compare_shares(df1, df2, i, alpha = .05):

    # критический уровень статистической значимости передаем из вне  или по умолчанию
    #alpha = .05 # критический уровень статистической значимости

    successes = np.array([df1['user_id'][i], df2['user_id'][i]]) #успехи, в нашем случае количество клиентов совершивших событие df2['event_name'][i] в группах А1 и А2 соответственно
    trials = trials = np.array([df1['user_id'][0], df2['user_id'][0]])    #испытания, в нашем случае количество клиентов на предыдущем шаге df2['event_name'][i-1] в группах А1 и А2 соответственно


    # пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]

    # пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]

    print(successes[0], successes[1], trials[0], trials[1])

    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])

    # разница пропорций в датасетах
    difference = p1 - p2

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / math.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = stats.norm(0, 1)

    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-значение: ', p_value)

    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница в событии ', df2['event_name'][i])
    else:
        print(
        'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными в событии ', df2['event_name'][i]
    )
    return

In [ ]:
conv_b

,event_name,user_id,ratio,rang
0,login,812,1.000000,1
2,product_page,461,0.567734,2
1,product_cart,225,0.277094,3
3,purchase,228,0.280788,4


In [ ]:
conv_a

,event_name,user_id,ratio,rang
0,login,2422,1.000000,1
2,product_page,1571,0.648637,2
1,product_cart,728,0.300578,3
3,purchase,773,0.319158,4


In [ ]:
# сводные таблицы conv_a, conv_b
for i in [1, 2, 3]:
    compare_shares(conv_a, conv_b, i, alpha = .05)

728 225 2422 812
p-значение:  0.20397790908072166
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными в событии  product_cart
1571 461 2422 812
p-значение:  3.651974405083358e-05
Отвергаем нулевую гипотезу: между долями есть значимая разница в событии  product_page
773 228 2422 812
p-значение:  0.04068470890052289
Отвергаем нулевую гипотезу: между долями есть значимая разница в событии  purchase


### Выводы и наблюдения

z-тест на пропорции (конверсии) увидел разницу между контрольной и экспериментальной группой в конверсиях события  `product_page` и `purchase`, но не увидел разницы в конверсии событий `product_cart`.

Доверие к результатам этого z-теста должно быть минимально - слишком недостоверные и ограниченные данные остались после отбрасывания всех неверных событий и пользователей. А группы сильно неравномерны - оличаются по количеству пользователей.

## Общий вывод по учебной работе

В учебной работе оценивалась корректность проведения А/В теста и анализировались результаты этого теста.

Проводилась работа по предъобработке и исследовательскому анализу данных, а также очистке данных и приведения их к требованиям технического задания.

Подготовка предоставленных  данных для тестирования была ранее проведена недостаточно хорошо. Были выявлены две группы ошибок.
1. **Ошибки фатальные** (их невозможно исправить не собрав данные заново)
        
        - время сбора информации не соответствует ТЗ. Сбор информации по событиям прекращен раньше даты, указанной в ТЗ;
        - количество и пропорции участников теста меньше указанного в ТЗ;
        - проведение маркетингового мероприятия во время сбора информации для теста.
        
2. **Ошибки, которые можно исправить** за счет отбрасывания недостоверных данных (это возможно увеличивает достоверность информации, но уменьшает количество данных)

        - в выборку попали посетители, участвующие в других тестах - влияние других показов может внести изменения в их поведение.
        - контрольная и эксперементальная группа имели пересечение пользователей;
        - контрольная и эксперементальная группа имеют большую разницу в количестве участников;
        - формулировка критериев (критерии сформулированы таким образом, что в 2х из 3х метрик количество выборки влияет на результат);
        - переформулировка метрик "на ходу" может субъективно повлиять на принятие решений.

На основании вышеизложенного
- данные предложенные для теста малоинформативны;
- результатам теста доверять нельзя.

In [ ]:
2+2

4